IMPORTS

In [ ]:
import numpy as np
from Scraping_script import get_address_info
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from datetime import datetime
import nest_asyncio
import asyncio
import seaborn as sns
from zksync_sdk import HttpJsonRPCTransport, ZkSyncProviderV01, network
nest_asyncio.apply()
from web3 import Web3
le = LabelEncoder()

Assigning Web3 Providers

In [ ]:
alchemy_websocket_std = 'insert websocket provider link'
w3_eth = Web3(Web3.WebsocketProvider(alchemy_websocket_std,websocket_timeout=None))
print(w3_eth.isConnected())

alchemy_websocket_polygon = 'insert websocket provider link'
w3_poly = Web3(Web3.WebsocketProvider(alchemy_websocket_polygon,websocket_timeout=None))
print(w3_poly.isConnected())

quicknode_websocket_rinkeby = 'insert websocket provider link'
w3_rk = Web3(Web3.WebsocketProvider(alchemy_websocket_polygon,websocket_timeout=None))
print(w3_rk.isConnected())

abi = '[{"inputs":[],"name":"decimals","outputs":[{"internalType":"uint8","name":"","type":"uint8"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"description","outputs":[{"internalType":"string","name":"","type":"string"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint80","name":"_roundId","type":"uint80"}],"name":"getRoundData","outputs":[{"internalType":"uint80","name":"roundId","type":"uint80"},{"internalType":"int256","name":"answer","type":"int256"},{"internalType":"uint256","name":"startedAt","type":"uint256"},{"internalType":"uint256","name":"updatedAt","type":"uint256"},{"internalType":"uint80","name":"answeredInRound","type":"uint80"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"latestRoundData","outputs":[{"internalType":"uint80","name":"roundId","type":"uint80"},{"internalType":"int256","name":"answer","type":"int256"},{"internalType":"uint256","name":"startedAt","type":"uint256"},{"internalType":"uint256","name":"updatedAt","type":"uint256"},{"internalType":"uint80","name":"answeredInRound","type":"uint80"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"version","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"}]'


web3 = Web3(Web3.WebsocketProvider(alchemy_websocket_std))

eth_usd = '0x5f4eC3Df9cbd43714FE2740f5E3616155c5b8419'

# Set up contract instance
contract = web3.eth.contract(address=eth_usd, abi=abi)


provider = ZkSyncProviderV01(provider=HttpJsonRPCTransport(network=network.mainnet))



In [ ]:
#Loading up dataset
sybil_data = pd.read_csv('data/hackathon-contributions-dataset_v2.csv')

In [ ]:
#Previewing Data
sybil_data.head(3)

In [ ]:
sybil_data.shape

In [ ]:
sybil_data.info()

In [ ]:
#Removing instances of missing addresses or chains
sybil_data.dropna(inplace=True)

In [ ]:
sybil_data.token.unique()

In [ ]:
sybil_data.chain.unique()

In [ ]:
#Label encoding tokens
sybil_data = sybil_data.assign(token = le.fit_transform(sybil_data["token"]))

In [ ]:
#Feature Selection
sybil_data = sybil_data[['address','grant_id','txn_hash','timestamp','chain','network','token']]

#Duplicates removing
sybil_data.drop_duplicates(inplace = True)

In [ ]:
#Counting the grants Funded
sybil_data['No_Grants_Funded'] = sybil_data.groupby('address')['address'].transform('count')

In [ ]:
#Feature restructuring
Feature_Columns = sybil_data.drop(['txn_hash'],axis = 1)

In [ ]:
#Getting unique addresses by removing instances of duplicate addresses
Feature_Columns.drop_duplicates(subset=['address'], inplace = True)

In [ ]:
#Separating addresses by chain
eth_std_rinkeby_addresses = list(Feature_Columns['address'].loc[(Feature_Columns['network']=='rinkeby')&(Feature_Columns['chain']=='eth_std')])
eth_std_mainnet_addresses = list(Feature_Columns['address'].loc[(Feature_Columns['network']=='mainnet')&(Feature_Columns['chain']=='eth_std')])
eth_polygon_addresses = list(Feature_Columns.address.loc[Feature_Columns['chain']=='eth_polygon'])
eth_zksync_addresses = list(Feature_Columns.address.loc[Feature_Columns['chain']=='eth_zksync'])


In [ ]:
#Calling the Data fetching function on the addresses
#Note: Due to the volume of address, this cell may take a long time to operate.
get_address_info(eth_std_mainnet_addresses,eth_std_rinkeby_addresses,eth_polygon_addresses,eth_zksync_addresses)

In [ ]:
#Loading the address information
address_info = pd.read_csv('data/address_info.csv')

In [ ]:
#Merging the address_information with the Preprocessed Gr_15 data
Features = pd.merge(Feature_Columns, address_info, on = "address", how = 'inner')

Allocating_points to Address & Classification of Addresses based on AcquiredPoints

In [ ]:
#Note: Due to the volume of address, this cell may take a long time to operate.

Acquired_points = 0
Funding_points = 0
transacting_points = 0
balance_points = 0
sybils_addresses = []
review_addresses = []


def point_allocator():
    for i in list(Features.address):
        Funding_count = int(Features.No_Grants_Funded.loc[Features['address']==i])
        no_transactions = int(Features.txn_count.loc[Features['address']==i])
        current_balance =  float(Features.balance_in_eth.loc[Features['address']==i])
        if Funding_count ==1:
            Funding_points =+1

        elif Funding_count <=4:
            Funding_points =+ 2

        else:
            Funding_points =3.33

        if no_transactions <=3:
            transacting_points =+1

        elif no_transactions <=5:
            transacting_points =+ 2

        else:
            transacting_points =+ 4.45

        if current_balance <=0.000547:
            balance_points =+1

        elif current_balance <=0.004721:
            balance_points =+ 2

        else:
            balance_points =+ 2.22

        Acquired_points = round(Funding_points+transacting_points+balance_points,2)
        if Acquired_points >= 6.6:
            pass
        elif Acquired_points >= 5.5 and Acquired_points <= 6.5:
            review_addresses.append(i)
        else:
            sybils_addresses.append(i)

        

point_allocator()

In [ ]:
#Creating textfiles for review and sybil addresses

with open('review_addresses', 'w') as ra: #Review
    for item in review_addresses:
        ra.write("%s\n" % item)
    print('Done')
ra.close()

with open('sybil_addresses', 'w') as sa: #Sybil
    for item in sybils_addresses:
        sa.write("%s\n" % item)
    print('Done')
sa.close()

Real-time Point System

In [ ]:
def Point_System(addresses):
    Price_Feed = contract.functions.latestRoundData().call()[1] / 10**8
    for address in addresses:
        Acquired_points = 0
        Funding_points = 0
        transacting_points = 0
        balance_points = 0
        Funding_count = sybil_data['address'].loc[sybil_data['address'] == address].count()
        checksum_address = Web3.toChecksumAddress(address)   


        if address in list(Feature_Columns['address'].loc[Feature_Columns['chain']=='eth_std']):
            no_transactions = w3_eth.eth.get_transaction_count(checksum_address)
            current_balance = w3_eth.eth.get_balance(checksum_address)/10**18
            print('eth_std')


        elif address in list(Feature_Columns['address'].loc[Feature_Columns['chain']=='eth_polygon']):
            no_transactions = w3_poly.eth.get_transaction_count(checksum_address)
            current_balance = w3_poly.eth.get_balance(checksum_address)/10**18
            print('eth_polygon')


        else:                    
            async def zksync(zk_addresses):               
                no_transactions = await provider.get_account_nonce(address=address)
                account_state = (await provider.get_state(address))
                if no_transactions == 0:
                    no_transactions = w3_eth.eth.get_transaction_count(checksum_address)
                    current_balance = w3_eth.eth.get_balance(checksum_address)/10**18
                    print('eth_std_to_zksync')

                    if no_transactions == 0:
                        no_transactions = w3_poly.eth.get_transaction_count(checksum_address)
                        current_balance = w3_poly.eth.get_balance(checksum_address)/10**18
                        print('eth_poly_to_zksync')

                    else:
                        pass

                else:
                    pass

                
                if 'ETH' and 'USDT' and 'USDC' in account_state.verified.balances.keys():
                    balance_in_USDT = account_state.verified.balances['USDT']/10**6
                    balance_in_USDC = account_state.verified.balances['USDC']/10**6
                    balance_in_gwei = account_state.verified.balances['ETH']/ 10**18
                    USDT_to_Eth = balance_in_USDT/Price_Feed
                    USDC_to_Eth = balance_in_USDC/Price_Feed
                    current_balance = balance_in_gwei+USDT_to_Eth+USDC_to_Eth
                    print('zksync') 
                elif 'ETH' and 'USDC' in account_state.verified.balances.keys():
                    balance_in_USDC = account_state.verified.balances['USDC']/10**6
                    balance_in_gwei = account_state.verified.balances['ETH']/ 10**18
                    USDC_to_Eth = balance_in_USDC/Price_Feed
                    current_balance = balance_in_gwei+USDC_to_Eth
                    print('zksync') 

                elif 'ETH' and 'USDT' in account_state.verified.balances.keys():
                    balance_in_USDT = account_state.verified.balances['USDT']/10**6
                    balance_in_gwei = account_state.verified.balances['ETH']/ 10**18
                    USDT_to_Eth = balance_in_USDT/Price_Feed
                    current_balance = balance_in_gwei+USDT_to_Eth
                    print('zksync') 

                elif 'USDC' in account_state.verified.balances.keys():
                    balance_in_USDC = account_state.verified.balances['ETH']/10**6
                    current_balance = balance_in_USDC/Price_Feed
                    print('zksync') 

                elif 'ETH' in account_state.verified.balances.keys():
                    balance_in_gwei = account_state.verified.balances['ETH']
                    current_balance = balance_in_gwei/ 10**18
                    print('zksync') 
                elif 'USDT' in account_state.verified.balances.keys():
                    balance_in_USDT = account_state.verified.balances['USDT']/10**6
                    current_balance = balance_in_USDT/Price_Feed
                    print('zksync') 
                else:
                    current_balance = 0
                return no_transactions,current_balance
            no_transactions,current_balance = asyncio.run(zksync(address))

        if Funding_count ==1:
            Funding_points =+1

        elif Funding_count <=4:
            Funding_points =+ 2

        else:
            Funding_points =3.33

        if no_transactions <=3:
            transacting_points =+1

        elif no_transactions <=5:
            transacting_points =+ 2

        else:
            transacting_points =+ 4.45

        if current_balance <=0.000547:
            balance_points =+1

        elif current_balance <=0.004721:
            balance_points =+ 2

        else:
            balance_points =+ 2.22

        Acquired_points = round(Funding_points+transacting_points+balance_points,2)
        if Acquired_points >= 6.6:
            print('Non_Sybil')
        elif Acquired_points >= 5.5 and Acquired_points <= 6.5:
            print('Needs human Review')
        else:
            print('sybil')




Point_System(addresses)#Enter an array of addresses